In [1]:
pip install rasterio

     |████████████████████████████████| 18.2MB 1.3MB/s 


In [0]:
import os
import numpy as np
import imageio
import imageio as im
import gdal
import rasterio 
from glob import glob
import matplotlib.pyplot as plt

In [0]:
## CONSTANTES PARA CALIBRAÇÃO RADIOMÉTRICA ##
MULT = 'MULT_BAND_10'
def encontrar(MULT):
    with open('/content/LC08_L1TP_220081_20180314_20180320_01_T1_MTL.txt', 'r') as MTL:
        linhas = MTL.readlines()
        for i in range(len(linhas)):
            if 'MULT_BAND_10' in linhas[i]:
                a = linhas[i].split('=')[-1].strip('\n')
                
    return a
ML = encontrar(MULT)

################################################################################################################################

ADD = 'ADD_BAND_10'
def encontrar(ADD):
    with open('/content/LC08_L1TP_220081_20180314_20180320_01_T1_MTL.txt', 'r') as MTL:
        linhas = MTL.readlines()
        for i in range(len(linhas)):
            if 'ADD_BAND_10' in linhas[i]:
                b = linhas[i].split('=')[-1].strip('\n')
                
    return b
AL = encontrar(ADD)

In [0]:
## CONSTANTES PARA CALCULAR TEMPERATURA DE BRILHO ##
K1 = 'K1_CONSTANT_BAND_10'
def encontrar(K1):
    with open('/content/LC08_L1TP_220081_20180314_20180320_01_T1_MTL.txt', 'r') as MTL:
        linhas = MTL.readlines()
        for i in range(len(linhas)):
            if 'K1_CONSTANT_BAND_10' in linhas[i]:
                b = linhas[i].split('=')[-1].strip('\n')
                
    return b
K1 = encontrar(K1)

################################################################################################################################

K2 = 'K2_CONSTANT_BAND_10'
def encontrar(K1):
    with open('/content/LC08_L1TP_220081_20180314_20180320_01_T1_MTL.txt', 'r') as MTL:
        linhas = MTL.readlines()
        for i in range(len(linhas)):
            if 'K2_CONSTANT_BAND_10' in linhas[i]:
                b = linhas[i].split('=')[-1].strip('\n')
                
    return b
K2 = encontrar(K1)

In [0]:
window = rasterio.windows.Window(2720, 3132, 1, 1)
window

Window(col_off=2720, row_off=3132, width=1, height=1)

In [0]:
banda10 = glob('/content/LC08_L1TP_220081_20180314_20180320_01_T1_B10.TIF')
with rasterio.open(banda10[0]) as src:
    b10= src.read(1, window=window)
b_10 = b10.astype(float)

NameError: ignored

In [0]:
print(src.crs)

EPSG:32622


In [0]:
banda10 = glob('/content/LC08_L1TP_220081_20180314_20180320_01_T1_B10.TIF')
with rasterio.open(banda10[0]) as src:
    b10= src.read(1)
b_10 = b10.astype(float)
kwargs = src.meta.copy()
kwargs.update({'dtype': 'float64'})

In [0]:
kwargs

{'count': 1,
 'crs': CRS.from_epsg(32622),
 'driver': 'GTiff',
 'dtype': 'float64',
 'height': 7841,
 'nodata': None,
 'transform': Affine(30.0, 0.0, 497685.0,
       0.0, -30.0, -3236085.0),
 'width': 7771}

In [0]:
#Calibração radiométrica
ml = float(ML)
al = float(AL)
def radcal(b_10):
    for pixel in b_10:
        radiance = (ml*b_10) + al
    return radiance
Lsens = radcal(b_10).astype(float)

with rasterio.open('radiance_b10.tiff', 'w', **kwargs) as dst:
    dst.write(Lsens.astype(rasterio.float64),1)

In [0]:
Lsens 

array([[9.8456062]])

In [0]:
#rad para temperatura de brilho
k1 = float(K1)
k2 = float(K2)
def TbTOA(Lsens):
    for i in Lsens:
        TbTOA = k2/(np.log((k1/Lsens)+1))
    return TbTOA

TbTOA = TbTOA(Lsens)

with rasterio.open('TbTOA.tiff', 'w', **kwargs) as dst:
    dst.write(TbTOA.astype(rasterio.float64),1)

In [0]:
TbTOA

array([[301.73222463]])

In [0]:
#Definição de variáveis conhecidas#


#a = open('vertical_profile.txt', 'r') #importa dados do perfil vertical
#VP = a.readlines()
P = 1010.4 #float(VP[5][1:7]) #pressão 
To = 26.1 + 273.15 #float(VP[5][17:21]) + 273.15 #temperatura próxima à superfície; temperatura da primeira camada do perfil vertical (K)
RH = 0.67 #float(VP[5][33:35])/100 #umidade relativa em fração

#Cálculo Vapor d'água método de Leckner (1978) "
Ps = np.exp(26.23 -(5416/To)) #cálculo da pressão parcial de vapor d’água 
w = (0.493*RH*Ps)/To  #cálculo do vapor d'água em g.cm**-2

#Cálculo Vapor d'água outro método#
#w0 = (1.0007+3.46*10**(-6)*P)*(6.1121)*np.exp(17.502*To/(240.97+To))
#w = w0*RH*0.098

#w = 0.0981*(10*0.6108*np.exp(17.27*(To-273.15)/237.3+(To-273.15))*RH)+0.1697

Ta =  16.011 + 0.9262*To  #temperatura do ar próxima à superfície

#variáveis conhecidas
emis = 0.9733 #emissivity
lamb = 10.904 #comprimento de onda efetivo para banda 10 sensor TIR LANDSAT 8 (Jiménez-Muñoz, 2014)
c1= 1.19104*10**8 #Wµm4m-2sr-1 
c2= 14387.7 #µmK

# FUNÇÕES ATMOSFÉRICAS BARSI #
Lu = 3.71
Ld = 5.59
trans = 0.54

# Gama e sigma #
gama = (((c2*Lsens)/(TbTOA*TbTOA))*((((lamb**4)/c1)*Lsens)+(1/lamb)))**-1
sigma = -(gama*Lsens) + TbTOA

In [0]:
### LST PELA INVERSÃO DA RTE ###
def temp_RTE(Lsens):
    for i in Lsens:
        a = c1/((lamb**5)*(((Lsens-Lu)/(emis*trans))-((1-emis)/emis)*Ld))
        b = (np.log(a+1))**-1
        temp = (c2/lamb)*b
    return temp

temp_RTE = temp_RTE(Lsens).astype(float)

with rasterio.open('Temp_RTE.tiff', 'w', **kwargs) as dst:
    dst.write(temp_RTE.astype(rasterio.float64),1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log
  """


In [0]:
temp_RTE

array([[312.63611926]])

In [12]:
### LST PELO SC GENERALIZADO (Temp_SC) ###
def tempSC(Lsens):
    # FUNÇÕES ATMOSFÉRICAS #
    psi1 = 0.04019*w*w + 0.02916*w + 1.01523
    psi2 = -0.38333*w*w - 1.50294*w + 0.20324
    psi3 = 0.00918*w*w + 1.36072*w - 0.27514
    
    for i in Lsens:
        tempSC = gama*(emis**-1*(psi1*Lsens + psi2) + psi3)+ sigma
    return tempSC

temperature_SC = tempSC(Lsens).astype(float)

with rasterio.open('temperature_SC.tiff', 'w', **kwargs) as dst:
    dst.write(temperature_SC.astype(rasterio.float64),1)

AttributeError: ignored

In [0]:
tempSC

array([[308.96255068]])

In [0]:
### LST PELO SC MELHORADO (Temp_ISC) ###
def tempISC(Lsens):
     # FUNÇÕES ATMOSFÉRICAS #
    psi1 = -7.2121979375*w*w+0.0000492124*Ta*Ta-2.4523205637*w-0.0262745276*Ta-0.0000496173*Ta*Ta*w+0.0231691781*Ta*w+0.0466282124*Ta*w*w-0.0000748260*Ta*Ta*w*w+4.4729730361
    psi2 = 89.6156888857*w*w-0.0003760208*Ta*Ta+106.5509303783*w+0.2157797227*Ta+0.0014080695*Ta*Ta*w-0.7844419527*Ta*w-0.5731956714*Ta*w*w+0.0009118768*Ta*Ta*w*w-30.3702785256
    psi3 = -14.6595491055*w*w-0.0001047275*Ta*Ta-79.9583806096*w+0.0418090158*Ta-0.0009095018*Ta*Ta*w+0.5453487543*Ta*w+0.0911362208*Ta*w*w-0.0001417749*Ta*Ta*w*w-3.7618398628
         
    for i in Lsens:
        tempISC = (gama*((((psi1*Lsens) + psi2)/emis) + psi3)) + sigma
    return tempISC

temperature_ISC = tempISC(Lsens).astype(float)

with rasterio.open('temperature_ISC.tiff', 'w', **kwargs) as dst:
    dst.write(temperature_ISC.astype(rasterio.float64),1)

In [0]:
tempISC

array([[313.59731675]])